# Regression

So far, in our exploration of machine learning, we have build sysems that predict discrete values: 

* whether a particular plant is Iris Setosa, Iris Versicolour, or Iris Virginica
* whether a person has diabetes or not
* whether a tumor is malignant or benign 


Even deciding whether an image is of a particular digit:

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/MnistExamples.png)

or which of 1,000 categories does a picture represent.


![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/poodleSmall.jpg)


This lab looks at how we can build classifiers that predict **continuous** values and such classifiers are called regression classifiers.

First, let's take a small detour into correlation.

## Correlation
A correlation is the degree of association between two variables. One of my favorite books on this topic is: 


![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/mg_statistics_big.png)



and they illustrate it by looking at
## Ladies expenditures on clothes and makeup
![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/mangaCorrelation1.png)
![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/mangeCorrelation2.png)



So let's go ahead and create that data in Pandas and show the table:

In [ ]:
import pandas as pd
from pandas import DataFrame
makeup =  [3000, 5000, 12000, 2000, 7000,  15000,  5000,  6000, 8000,  10000]
clothes = [7000, 8000, 25000, 5000, 12000, 30000, 10000, 15000, 20000, 18000]
ladies = ['Ms A','Ms B','Ms C','Ms D','Ms E','Ms F','Ms G','Ms H','Ms I','Ms J',]
monthly = DataFrame({'makeup': makeup, 'clothes': clothes}, index= ladies)
monthly

and let's show the scatterplot

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
x = figure(title="Montly Expenditures on Makeup and Clothes", x_axis_label="Money spent on makeup", y_axis_label="Money spent on clothes")
x.circle(monthly['makeup'], monthly['clothes'], size=10, color="navy", alpha=0.5)
output_file("stuff.html")

show(x)

When the data points are close to a straight line going up, we say that there is a positive correlation between the two variables. So in the case of the plot above, it visually looks like a postive correlation. Let's look at a few more examples:

## Weight and calories consumed in 1-3 yr/old children
This small but real dataset examines whether young children who weigh more, consume more calories


In [ ]:
weight = [7.7, 7.8, 8.6, 8.5, 8.6, 9, 10.1, 11.5, 11, 10.2, 11.9, 10.4, 9.3, 9.1, 8.5, 11]
calories = [360, 400, 500, 370, 525, 800, 900, 1200, 1000, 1400, 1600, 850, 575, 425, 950, 800]
kids = DataFrame({'weight': weight, 'calories': calories})
kids

In [ ]:
p = figure(title="Weight and calories in 1-3 yr.old children", 
           x_axis_label="weight (kg)", y_axis_label='weekly calories')
p.circle(kids['weight'], kids['calories'], size=10, color='navy', alpha=0.5)
show(p)


And again, there appears to be a positive correlation.

## The stronger the correlation the closer to a straight line
The closer the data points are to a straight line, the higher the correlation. A rising straight line (rising going left to right) would be perfect positive correlation. Here we are comparing the heights in inches of some NHL players with their heights in cm. Obviously, those are perfectly correlated.

In [ ]:
inches =[68, 73, 69,72,71,77]
cm = [173, 185, 175, 183, 180, 196]
nhlHeights = DataFrame({'heightInches': inches, 'heightCM': cm})
nhlHeights

In [ ]:
p = figure(title="Comparison of Height in Inches and Height in CM", 
           x_axis_label="Height in Inches", 
           y_axis_label="Height in centimeters")
p.circle(nhlHeights['heightInches'], nhlHeights['heightCM'],
         size=10, color='navy', alpha=0.5)
show(p)

## No correlation = far from straight line
On the opposite extreme, if the datapoints are scattered and no line is discernable, there is no correlation.

Here we are comparing length of the player's hometown name to his height in inches. We are checking whether a player whose hometown name has more letters, tends to be taller. For example, maybe someone from Medicine Hat is taller than someone from Ledue. Obviously there should be no correlation. 


(Again, a small but real dataset)

In [ ]:
medicineHat = pd.read_csv('https://raw.githubusercontent.com/zacharski/machine-learning-notebooks/master/data/medicineHatTigers.csv')
medicineHat['hometownLength'] = medicineHat['Hometown'].str.len()
medicineHat

In [ ]:
p = figure(title="Correlation of the number of Letters in the Hometown to Height", 
           x_axis_label="Player's Height", y_axis_label="Hometown Name Length")
p.circle(medicineHat['Height'], medicineHat['hometownLength'], size=10, color='navy', alpha=0.5)
show(p)

And that does not look at all like a straight line. 

## negative correlation has a line going downhill
When the slope goes up, we say there is a positive correlation and when it goes down there is a negative correlation.

#### the relationship of hair length to a person's height

In [ ]:
height =[62, 64, 65, 68, 69, 70, 67, 65, 72, 73, 74]
hairLength = [7, 10, 6, 4, 5, 4, 5, 8, 1, 1, 3]
cm = [173, 185, 175, 183, 180, 196]
people = DataFrame({'height': height, 'hairLength': hairLength})
p = figure(title="Correlation of hair length to a person's height", 
           x_axis_label="Person's Height", y_axis_label="Hair Length")
p.circle(people['height'], people['hairLength'], size=10, color='navy', alpha=0.5)
show(p)

There is a strong negative correlation between the length of someone's hair and how tall they are. Why do you think that is?


![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)


# Numeric Representation of the Strength of the Correlation

So far, we've seen a visual representation of the correlation, but we can also represent the degree of correlation numerically.

## Pearson Correlation Coefficient

This ranges from -1 to 1.
1 is perfect positive correlation, -1 is perfect negative.

$$r=\frac{\sum_{i=1}^n(x_i - \bar{x})(y_i-\bar{y})}{\sqrt{\sum_{i=1}^n(x_i - \bar{x})}  \sqrt{\sum_{i=1}^n(y_i - \bar{y})}}$$

In Pandas it is very easy to compute.

### Japanese ladies expenses on makeup and clothes
Let's go back to our first example.
First here is the data:

In [ ]:
monthly

In [ ]:
p = figure(title="Montly Expenditures on Makeup and Clothes", 
           x_axis_label="Money spent on makeup", y_axis_label="Money spent on clothes")
p.circle(monthly['makeup'], monthly['clothes'], size=10, color='navy', alpha=0.5)
show(p)


So that looks like a pretty strong positive correlation. To compute Pearson on this data we do:

In [ ]:
monthly.corr()

There is no surprise that makeup is perfectly correlated with makeup and clothes with clothes (those are the 1.000 on the diagonal). The interesting bit is that the Pearson for makeup to clothes is 0.968. That is a pretty strong correlation. 

If you are interesting you can compute the Pearson values for the datasets above, but let's now move to ...


![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)


## Regression

Let's say we know a young lady who spends about  ¥22,500 per month on clothes (that's about $200/month). What do you think she spends on makeup, based on the chart below?

In [ ]:
show(p)

I'm guessing you would predict she spends somewhere around  ¥10,000 a month on makeup (almost $100/month). And how we do this is when looking at the graph  we mentally draw an imaginary straight line through the datapoints and use that line for predictions. We are performing human linear regression.  And as humans, we have the training set--the dots representing data points on the graph. and we **fit** our human classifier by mentally drawing that straight line. That straight line is our model. Once we have it, we can throw away the data points. When we want to make a prediction, we see where the money spent on clothes falls on that line. 

We just predicted a continuous value (money spent on makeup) from one factor (money spent on clothes).

## Predicting MPG

What happens when we want to predict a continuous value from 2 factors? Suppose we want to predict MPG based on the weight of a car and its horsepower.

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/regression.png)
from [Mathworks](https://www.mathworks.com/help/stats/regress.html)

Now instead of a line representing the relationship we have a plane.

Let's create a linear regression classifier and try this out!

First, let's get the data.


In [ ]:
columnNames = ['mpg', 'cylinders', 'displacement', 'HP', 'weight', 'acceleration', 'year', 'origin', 'model']
cars = pd.read_csv('https://raw.githubusercontent.com/zacharski/ml-class/master/data/auto-mpg.csv',
                   na_values=['?'], names=columnNames)
cars = cars.set_index('model')
cars = cars.dropna()
cars

Now divide the dataset into training and testing. And let's only use the horsepower and weight columns as features.

In [ ]:
from sklearn.model_selection import train_test_split
cars_train, cars_test = train_test_split(cars, test_size = 0.2)
cars_train_features = cars_train[['HP', 'weight']]
# cars_train_features['HP'] = cars_train_features.HP.astype(float)
cars_train_labels = cars_train['mpg']
cars_test_features = cars_test[['HP', 'weight']]
# cars_test_features['HP'] = cars_test_features.HP.astype(float)
cars_test_labels = cars_test['mpg']

cars_test_features

### SKLearn Linear Regression

Now let's create a Linear Regression classifier and fit it.

In [ ]:
from sklearn.linear_model import LinearRegression
linclf = LinearRegression()
linclf.fit(cars_train_features, cars_train_labels)

and finally use the trained classifier to make predictions on our test data

In [ ]:
predictions = linclf.predict(cars_test_features)

Let's take an informal look at how we did:

In [ ]:
results = cars_test_labels.to_frame()
results['Predicted']= predictions
results

Here is what my output looked like:

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/predictMPG.png)

as you can see the first two predictions were pretty close as were a few others.




![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)

### Determining how well the classifier performed

With categorical classifiers we used sklearn's accuracy_score:

```
from sklearn.metrics import accuracy_score
```
Consider a task of predicting whether an image is of a dog or a cat. We have 10 instances in our test set. After our classifier makes predictions, for each image we have the actual (true) value, and the value our classifier predicted:

actual | predicted
:-- | :---
dog | dog
**dog** | **cat**
cat | cat
dog | dog
cat | cat
**cat** | **dog**
dog | dog
cat | cat
cat | cat
dog | dog

sklearn's accuracy score just counts how many predicted values matched the actual values and then divides by the total number of test instances. In this case the accuracy score would be .8000. The classifier was correct 80% of the time. 

We can't use this method with a regression classifier. In the image above, the actual MPG of the Peugeot 304 was 30 and our classifier predicted 30.038619. Does that count as a match or not? The actual mpg of a Pontiac Sunbird Coupe was 24.5 and we predicted 25.57. Does that count as a match? Instead of accuracy_score, there are different evaluation metrics we can use. 

#### Mean Squared Error and Root Mean Square Error

A common metric is Mean Squared Error or MSE. MSE is a measure of the quality of a regression classifier. The closer MSE is to zero, the better the classifier. Let's look at some made up data to see how this works:

vehicle | Actual MPG | Predicted MPG
:---: | ---: | ---: 
Ram Promaster 3500 | 18.0 | 20.0
Ford F150 | 20 | 19
Fiat 128 | 33 | 33

First we compute the error (the difference between the predicted and actual values)

vehicle | Actual MPG | Predicted MPG | Error
:---: | ---: | ---: | --:
Ram Promaster 3500 | 18.0 | 20.0 | -2
Ford F150 | 20 | 19 | 1
Fiat 128 | 33 | 33 | 0

Next we square the error and compute the average:

vehicle | Actual MPG | Predicted MPG | Error | Error^2
:---: | ---: | ---: | --: | ---:
Ram Promaster 3500 | 18.0 | 20.0 | -2 | 4
Ford F150 | 20 | 19 | 1 | 1
Fiat 128 | 33 | 33 | 0 | 0
MSE | - | - | - | 1.667

**Root Mean Squared Error** is simply the square root of MSE. The advantage of RMSE is that it has the same units as what we are trying to predict. Let's take a look ...

In [ ]:
from sklearn.metrics import mean_squared_error
MSE  = mean_squared_error(cars_test_labels, predictions)
RMSE = mean_squared_error(cars_test_labels, predictions, squared=False)
print("MSE: %5.3f.   RMSE: %5.3f" %(MSE, RMSE))

That RMSE tells us on average how many mpg we were off.






![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/torchdivide.png)

# <font color='#EE4C2C'>You Try ... </font> 
## <font color='#EE4C2C'>Predicting the range of electric vehicles</font> 

We are wondering whether we can predict the range of an EV from its battery capacity and efficiency. It seems like it might. If our battery holds 60 kWh (kilowatt hours), and it it takes 200 wh (watt hours) to go 1 kilometer, you might predict that we could go 60,000 / 200 = 300 kilometers on a charge. This is equivalent to if you know the capacity of your gas tank and how efficient (MPG) your car is you can roughly predict the range of your car. For example, if the tank capacity is 10 gallons and it gets 30 MPG, you should be able to go 300 miles.

The data is in the file 

[https://raw.githubusercontent.com/zacharski/ml-class/master/data/ElectricCarData_w_battery.csv](https://raw.githubusercontent.com/zacharski/ml-class/master/data/ElectricCarData_w_battery.csv)

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>1. Load that file into a Pandas DataFrame</font> 

In [ ]:
#TODO

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>2. Divide into training and test datasets</font> 
Now divide the dataset into training and testing. And let's only use the `Efficiency_WhKm`, `battery_kWh` columns as features and the column `Range_Km` is what we are trying to predict.

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>3. SKLearn Linear Regression</font> 

Now let's create a Linear Regression classifier and fit it.

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>4. Predictions</font>
Let's use the classifier to make predictions on our test dataset.

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>5. RMSE</font>
What is the Root Mean Square Error?

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>6. How well does the classifier work?</font>
Just double click this text cell and type in your answer.


![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)



## So what kind of model does a linear regression classifier build?

You probably know this if you reflect on grade school math classes you took. 

Let's go back and look at the young ladies expenditures on clothes and makeup.

In [ ]:
p = figure(title="Montly Expenditures on Makeup and Clothes", 
           x_axis_label="Money spent on makeup", y_axis_label="Money spent on clothes")
p.circle(monthly['makeup'], monthly['clothes'], size=10, color='navy', alpha=0.5)
show(p)

When we talked about this example above, I mentioned that when we do this, we imagine a line. Let's see if we can use sklearns linear regression classifier to draw that line:

In [ ]:
regr = LinearRegression()
regr.fit(monthly[['clothes']], monthly['makeup'])
pred = regr.predict(monthly[['clothes']])

import matplotlib.pyplot as plt
# Plot outputs
plt.scatter(monthly['clothes'], monthly['makeup'],  color='black')
plt.plot(monthly['clothes'], pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

Hopefully that matches your imaginary line!

The formula for the line is

$$makeup=w_0clothes + y.intercept$$

We can query our classifier for those values ($w_0$, and $y.intercept$):

In [ ]:
print('w0 = %5.3f' % regr.coef_)
print('y intercept = %5.3f' % regr.intercept_)

So the formula for this particular example is

$$ makeup = 0.479 * clothes + 121.782$$

So if a young lady spent ¥22,500 on clothes we would predict she spent the following on makeup:



In [ ]:
makeup = regr.coef_[0] * 22500 + regr.intercept_
makeup

The formula for regression in general is

$$\hat{y}=\theta_0 + \theta_1x_1 + \theta_2x_2 + ... \theta_nx_n$$

where $\theta_0$ is the y intercept.  When you fit your classifier it is learning all those $\theta$'s. That is the model your classifier learns. 

It is important to understand this as it applies to other classifiers as well!


![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)


## Overfitting
Consider two models for our makeup predictor. One is the straight line:

In [ ]:
plt.scatter(monthly['clothes'], monthly['makeup'],  color='black')
plt.plot(monthly['clothes'], pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

And the other looks like:

In [ ]:
monthly2 = monthly.sort_values(by='clothes')
plt.scatter(monthly2['clothes'], monthly2['makeup'],  color='black')
plt.plot(monthly2['clothes'], monthly2['makeup'], color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

The second model fits the training data perfectly. Is it better than the first?  Here is what could happen.  

Let's say we have been tuning our model using our validation data set. Our error rates look like 

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/overfitter.png)

As you can see our training error rate keeps going down, but at the very end our validation error increases. This is called **overfitting** the data. The model is highly tuned to the nuances of the training data. So much so, that it hurts the performance on new data--in this case, the validation data. This, obviously, is not a good thing.



![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)
#### An aside 

Imagine preparing for a job interview for a position you really, really, want. Since we are working on machine learning, let's say it is a machine learning job. In their job ad they list a number of things they want the candidate to know:

* Convolutional Neural Networks
* Long Short Term Memory models
* Recurrent Neural Networks
* Generative Deep Learning

And you spend all your waking hours laser focused on these topics. You barely get any sleep and you read articles on these topics while you eat. You know the tiniest intricacies of these topics. You are more than 100% ready.

The day of the interview arrives. After of easy morning of chatting with various people, you are now in a conference room for the technical interview, standing at a whiteboard, ready to hopefully wow them with your wisdom. The first question they ask is for you to write the solution to the fizz buzz problem:

> Write a program that prints the numbers from 1 to 100. But for multiples of three print “Fizz” instead of the number and for the multiples of five print “Buzz”. For numbers which are multiples of both three and five print “FizzBuzz”

And you freeze. This is a simple request and a very common interview question. In fact, to prepare you for this job interview possibility, write it now:





In [ ]:
def fizzbuzz():
  print("TO DO")

In [ ]:
fizzbuzz()

Back to the job candidate freezing, this is an example of overfitting. You overfitting to the skills mentioned in the job posting.

At dissertation defenses often faculty will ask the candidate questions outside of the candidate's dissertation. I heard of one case in a physics PhD defense where a faculty member asked "Why is the sky blue?" and the candidate couldn't answer. 

Anyway, back to machine learning.

---


There are a number of ways to reduce the likelihood of overfitting including

* We can reduce the complexity of the model. Instead of going with the model of the jagged line immediately above we can go with the simpler straight line model. We have seen this in decision trees where we limit the depth of the tree.

* Another method is to increase the amount of training data.

Let's examine the first. The process of reducing the complexity of a model is called regularization.

The linear regression model we have just used tends to overfit the data and there are some variants that are better and these are called regularized linear models. These include

* Ridge Regression
* Lasso Regression
* Elastic Net - a combination of Ridge and Lasso

Let's explore Elastic Net. And let's use all the columns of the car mpg dataset:

In [ ]:
newTrain_features = cars_train.drop('mpg', axis=1)
newTrain_labels = cars_train['mpg']
newTest_features = cars_test.drop('mpg', axis=1)
newTest_labels = cars_test['mpg']

First, let's try with our standard Linear Regression classifier:

In [ ]:
linclf = LinearRegression()
linclf.fit(newTrain_features, newTrain_labels)
predictions = linclf.predict(newTest_features)

In [ ]:
MSE  = mean_squared_error(newTest_labels, predictions)
RMSE = mean_squared_error(newTest_labels, predictions, squared=False)
print("MSE: %5.3f.   RMSE: %5.3f" %(MSE, RMSE))

Now let's try with [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)

In [ ]:
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(newTrain_features, newTrain_labels)
ePredictions = elastic_net.predict(newTest_features)

In [ ]:
MSE  = mean_squared_error(newTest_labels, ePredictions)
RMSE = mean_squared_error(newTest_labels, ePredictions, squared=False)
print("MSE: %5.3f.   RMSE: %5.3f" %(MSE, RMSE))



I've run this a number of times. Sometimes linear regression is slightly better and sometimes ElasticNet is.  Here are the results of one run:

##### RMSE

Linear Regression | Elastic Net
:---: | :---:
2.864 | 2.812

So this is not the most convincing example.

However, in general, it is always better to have some regularization, so (mostly) you should avoid the generic linear regression classifier.

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/torchdivide.png)

# <font color='#EE4C2C'>You Try ... Happiness</font> 
## <font color='#EE4C2C'>Better Life Index</font> 


What better way to explore regression then to look at happiness.

From a Zen perspective, happiness is being fully present in the current moment. 

But, ignoring that advice, let's see if we can predict happiness, or life satisfaction.

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/happy2.png)

We are going to be investigating the [Better Life Index](https://stats.oecd.org/index.aspx?DataSetCode=BLI). You can download a csv file of that data from that site. (under the *Export* menu select *Text File (CSV)*)

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/better2.png)

Now that you have the CSV data file on your laptop, you can upload it to Colab.

In Colab, you will see a list of icons on the left. 

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/uploadFile4.png)

Select the file folder icon. 

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/uploadFiles5b.png)

Next, select the upload icon (the page with an arrow icon). And upload the file.

Next, let's execute the Linux command `ls`:



In [ ]:
!ls

You will notice that the BLI file is now uploaded to Google Colab.

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>1. Load that file into a Pandas DataFrame</font> 

We will load the file into Pandas Dataframe called `bli` for better life index:

In [ ]:
# TO DO

bli = "TO DO"
bli

When examining the DataFrame we can see it has an interesting structure. So the first row we can parse:

* The country is Australia
* The feature is Labour market insecurity
* The Inequality column tells us it is the **total** Labour market insecurity value.
* The unit column tells the us the number is a percentage.
* And the value is 3.1 (this may be different if you have a newer version of the file than the one I used)

So, in English, the column is The total labor market insecurity for Australia is 5.40%.

I am curious as to what values other than Total are in the Inequality column:





In [ ]:
bli.Inequality.unique()

Cool. So in addition to the total for each feature, we can get values for just men, just women, and the high and low. 

Let's get just the totals and then pivot the DataFrame so it is in a more usable format. 

In addition, there are a lot of NaN values in the data, let's replace them with the mean value of the column.

We are just learning about regression and this is a very small dataset, so let's divide training and testing by hand ...

In [ ]:

bli = bli[bli["INEQUALITY"]=="TOT"]
bli = bli.pivot(index="Country", columns="Indicator", values="Value")
bli.fillna(bli.mean(), inplace=True)
bliTest = bli.loc['Greece':'Italy', :]
bliTrain = pd.concat([bli.loc[:'Germany' , :], bli.loc['Japan':, :]])
bliTrain

We are trying to predict Life satisfaction, where *life satisfaction* is defined as:

>The indicator considers people's evaluation of their life as a whole. It is a weighted-sum of different response categories based on people's rates of their current life relative to the best and worst possible lives for them on a scale from 0 to 10, using the Cantril Ladder (known also as the "Self-Anchoring Striving Scale").
“High”/ “Low” refer to values for people with tertiary/ below upper secondary education.

Let's get some basic information on life satisfaction from the dataset:

In [ ]:
bli['Life satisfaction'].describe()

So the indicator is a scale from 0 to 10, but the minimum satisfaction in our dataset was 4.9 and the happiest (i.e. life satisfaction) was 7.9. Let's find out which countries they were: {again you might have a newer version of the data)



In [ ]:
print("The least happy country is " + bli['Life satisfaction'].idxmin())
print("The happiest country is " + bli['Life satisfaction'].idxmax())

#### Now we need to divide both the training and test sets into features and labels. 

In [ ]:
feature_names   = bliTrain.columns.values.tolist()
feature_names.remove('Life satisfaction')
feature_names

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>2. Dividing the datasets into features and labels</font> 

Go ahead and construct:

* `bliTrain_features`
* `bliTrain_labels`
* `bliTest_features`
* `bliTest_labels`




In [ ]:
# TO DO




![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>3. Create and Train an elastic net model</font> 

In [ ]:
# TO DO




![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>4. Use the trained model to make predictions on our tiny test set</font> 

 

In [ ]:
# TO DO


Now let's visually compare the differences between the predictions and the actual values

How did you do? For me Iceland was a lot happier than what was predicted.

Now we will compute error:



# Prediction Housing Prices

## But first some wonkiness
When doing one hot encoding, sometimes the original datafile has the same type of data in multiple columns. For example...

Title | Genre 1 | Genre 2
 :--: | :---: | :---: 
 Mission: Impossible - Fallout | Action | Drama
 Mama Mia: Here We Go Again | Comedy | Musical
 Ant-Man and The Wasp | Action | Comedy
 BlacKkKlansman | Drama | Comedy
 
 
 When we one-hot encode this we get something like
 
 Title | Genre1 Action | Genre1 Comedy | Genre1 Drama | Genre2 Drama | Genre2 Musical | Genre2 Comedy
  :--: | :--: | :--: | :--: | :--: | :--: | :--: 
  Mission: Impossible - Fallout | 1 | 0 | 0 | 1 | 0 | 0
  Mama Mia: Here We Go Again  | 0 | 1 | 0 | 0 | 1 | 0
  Ant-Man and The Wasp | 1 | 0 | 0 | 0 | 0 | 1
  BlacKkKlansman | 0 | 0 | 1 | 0 | 0 | 1
  
  But this isn't what we probably want. Instead this would be a better representation:
  
  Title | Action | Comedy | Drama | Musical
  :---: | :---: | :---: |  :---: | :---: | 
  Mission: Impossible - Fallout | 1 | 0 | 1 | 0
  Mama Mia: Here We Go Again  | 0 | 1 | 0 | 1
  Ant-Man and The Wasp | 1 | 1 | 0 | 0
  BlacKkKlansman | 0 | 1 | 1 | 0
  
  Let's see how we might do this in code


In [ ]:
df   = pd.DataFrame({'Title': ['Mission: Impossible - Fallout', 'Mama Mia: Here We Go Again', 
                               'Ant-Man and The Wasp', 'BlacKkKlansman' ],
                    'Genre1': ['Action', 'Comedy', 'Action', 'Drama'],
                    'Genre2': ['Drama', 'Musical', 'Comedy', 'Comedy']})
df

In [ ]:
one_hot_1 = pd.get_dummies(df['Genre1'])
one_hot_2 = pd.get_dummies(df['Genre2'])

In [ ]:
# now get the intersection of the column names
s1 = set(one_hot_1.columns.values)
s2 = set(one_hot_2.columns.values)
intersect = s1 & s2
only_s1 = s1 - intersect
only_s2 = s2 - intersect
# now logically or the intersect
logical_or = one_hot_1[list(intersect)] | one_hot_2[list(intersect)]
# then combine everything
combined = pd.concat([one_hot_1[list(only_s1)], logical_or, one_hot_2[list(only_s2)]], axis=1)
combined

### Now drop the two original columns and add the one hot encoded columns
df= df.drop('Genre1', axis=1)
df= df.drop('Genre2', axis=1)
df = df.join(combined)
df

That looks more like it!!!

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/torchdivide.png)

# <font color='#EE4C2C'>The task: Predict Housing Prices</font> 



Your task is to create a regession classifier that predicts house prices. The data and a description of

* [The description of the data](https://raw.githubusercontent.com/zacharski/ml-class/master/data/housePrices/data_description.txt)
* [The CSV file](https://raw.githubusercontent.com/zacharski/ml-class/master/data/housePrices/data.csv)


Minimally, your classifier should be trained on the following columns:

In [ ]:
numericColumns = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
                 'FullBath', 'HalfBath', 'Bedroom', 'Kitchen']
categoryColumns = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 
                   'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'BldgType', 
                   'HouseStyle', 'RoofStyle', 'RoofMatl', '' ]

# Using multicolumns is optional
multicolumns = [['Condition1', 'Condition2'], ['Exterior1st', 'Exterior2nd']]

You are free to use more columns than these. Also, you may need to process some of the columns. 
 Here are the requirements:
 
 ### 1. Drop any data rows that contain Nan in a column. 
 Once you do this you should have around 1200 rows.
 ### 2. Use the following train_test_split parameters
  ```
train_test_split( originalData, test_size=0.20, random_state=42)
```

 ### 3. You are to compare Linear Regression and Elastic Net
 ### 4. You should use 10 fold cross validation (it is fine to use grid search)
 ### 5. When finished tuning your model, determine the accuracy on the test data using RMSE.

# Performance Bonus
You are free to adjust any hyperparameters but do so before you evaluate the test data. You may get up to 15xp bonus for improved accuracy. 

Good luck!